In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

In [ ]:
pos = 27297*2
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

In [ ]:
IMG_SIZE = 256

# Create a generator

data_augmentation_1 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.01, 0.01), fill_mode='reflect')
])

data_augmentation_2 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.005, 0.005), fill_mode='reflect')
])

## Augmentation

In [ ]:
BATCH_SIZE = 32
checkpoint_filepath = 'checkpoints/AUC/checkpoint_aug'
archi = 'Dnet121'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

In [ ]:
record_file_train = 'copd_train_new.tfrecords'
train_majority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases
                  
train_minority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

train_majority = train_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

train_dataset = train_majority.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_dataset.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_dataset.shuffle(2*pos+neg).batch(BATCH_SIZE)

In [ ]:
record_file_val = 'copd_val_new.tfrecords'
val_majority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases

val_minority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

val_majority = val_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

val_dataset = val_majority.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = val_dataset.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = val_dataset.shuffle(42908).batch(BATCH_SIZE)

In [ ]:
model = load_model_from_pretrain(archi)

metric = tf.keras.metrics.AUC(name='auc')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback)

gc.collect()

In [ ]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_aug'
archi = 'Dnet121'

model = define_model(archi)

model.load_weights(checkpoint_filepath)

In [ ]:
record_file = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.batch(32)) # filter out negative cases

y_preds = model.predict(test_dataset)

In [ ]:
y_test = get_data_label('test')

test_CI(y_preds, y_test)

print(average_precision_score(y_test, y_preds, average=None))
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))

gc.collect()